In [ ]:
!pip install torch
import numpy as np
np.random.seed(0)
import random

In [ ]:
import kagglehub
import torch
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Download latest version
path = kagglehub.dataset_download("bishals098/nasa-cmapss-2-engine-degradation")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'nasa-cmapss-2-engine-degradation' dataset.
Path to dataset files: /kaggle/input/nasa-cmapss-2-engine-degradation


In [ ]:
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd

In [ ]:
import kagglehub
import h5py
import numpy as np
import pandas as pd

# Assuming `path` is already defined from kagglehub.dataset_download(...)
h5_path = path + "/N-CMAPSS_DS01-005.h5"
print("Using HDF5 file:", h5_path)

# 1. Load TRAIN/DEV arrays and column names from HDF5
with h5py.File(h5_path, 'r') as hdf:
    # Main dev arrays (TRAIN DATA)
    A_dev  = hdf["A_dev"][:]      # (N, 4)  -> ['unit', 'cycle', 'Fc', 'hs']
    T_dev  = hdf["T_dev"][:]      # (N, 10) -> health modifiers
    W_dev  = hdf["W_dev"][:]      # (N, 4)  -> ['alt', 'Mach', 'TRA', 'T2']
    Xs_dev = hdf["X_s_dev"][:]    # (N, 14) -> physical sensors
    Xv_dev = hdf["X_v_dev"][:]    # (N, 14) -> virtual sensors
    Y_dev  = hdf["Y_dev"][:]      # (N, 1)  -> RUL target

    # Column names (decode from bytes -> str)
    A_var  = [x.decode("utf-8") for x in hdf["A_var"][:]]
    T_var  = [x.decode("utf-8") for x in hdf["T_var"][:]]
    W_var  = [x.decode("utf-8") for x in hdf["W_var"][:]]
    Xs_var = [x.decode("utf-8") for x in hdf["X_s_var"][:]]
    Xv_var = [x.decode("utf-8") for x in hdf["X_v_var"][:]]
gc.collect()
# 2. Stack all TRAIN feature blocks horizontally into one big matrix
X_all_train = np.hstack([A_dev, T_dev, W_dev, Xs_dev, Xv_dev])

# 3. Build full feature name list in the same order
feature_names = A_var + T_var + W_var + Xs_var + Xv_var

# 4. Create full TRAIN pandas DataFrame (includes unit, cycle, all features)
df_dev = pd.DataFrame(X_all_train, columns=feature_names)

# 5. Add RUL label as a new column
df_dev["RUL"] = Y_dev[:, 0]

# 6. Remove constant (useless) columns
constant_cols = [
    'fan_eff_mod', 'fan_flow_mod',
    'LPC_eff_mod', 'LPC_flow_mod',
    'HPC_eff_mod', 'HPC_flow_mod',
    'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod'
]

# Pandas TRAIN dataset
df_dev_clean = df_dev.drop(columns=constant_cols)

# forward fill data
df_dev_clean = df_dev_clean.ffill()

# 7. Define feature columns (drop RUL + unit)
#feature_cols = [c for c in df_dev_clean.columns if c not in ["RUL", "unit"]]
#feature_cols = [c for c in df_dev_clean.columns if c not in ["RUL"]]

# ---- PANDAS TRAIN SET ----
#X_train_df = df_dev_clean[feature_cols]   # features (no unit, includes cycle)
X_train_df = df_dev_clean
df_dev_clean['target'] = np.where(df_dev_clean["RUL"]>5, 1, 0)
y_train_df = df_dev_clean["target"]          # labels (RUL>5 --> 1; o/w, 0)

# take every 1000th row unless a failure row
failure_mask = (y_train_df == 0)
row_mask = (y_train_df.reset_index().index % 1000 == 0)
y_train_df = y_train_df[failure_mask | row_mask]
X_train_df = X_train_df[failure_mask | row_mask]

print(y_train_df.head)
print(y_train_df.shape)
#values_to_keep = np.where(condition_mask)[0]
#values_to_keep = np.where(y_test_np == 0)[0]
#X_train_df = X_train_df.iloc[::1000]
#y_train_df = y_train_df.iloc[::1000]

# ---- NUMPY TRAIN SET ----
no_unit_pd_x = X_train_df.drop(labels = ['unit', 'RUL', 'target'], axis=1)
print(f"no unit columns: {no_unit_pd_x.columns}")
# some feature selection
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model = RFC()
model.fit(no_unit_pd_x, y_train_df)
rf_importances = pd.Series(model.feature_importances_, index=no_unit_pd_x.columns)
n_features = 10
(rf_importances.nlargest(n_features).plot(kind='barh', title = 'Top Feature Importances RF'))
plt.show()

no_unit_pd_x_rf = no_unit_pd_x[rf_importances.nlargest(n_features).index[:10]]
no_unit_pd_x_svm = no_unit_pd_x[rf_importances.nlargest(n_features).index[:5]]
print(f"after feature selection: {no_unit_pd_x}")
X_train_np_rf = no_unit_pd_x_rf.to_numpy()
X_train_np_svm = no_unit_pd_x_svm.to_numpy()
y_train_np = y_train_df.to_numpy()

X_train_df = X_train_df.drop(columns = ['RUL', 'target'])
print(f"column labels: {X_train_df.columns}")

Using HDF5 file: /kaggle/input/nasa-cmapss-2-engine-degradation/N-CMAPSS_DS01-005.h5
<bound method NDFrame.head of 0          1
1000       1
2000       1
3000       1
4000       1
          ..
4906631    0
4906632    0
4906633    0
4906634    0
4906635    0
Name: target, Length: 348983, dtype: int64>
(348983,)
no unit columns: Index(['cycle', 'Fc', 'hs', 'HPT_eff_mod', 'alt', 'Mach', 'TRA', 'T2', 'T24',
       'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50',
       'Nf', 'Nc', 'Wf', 'T40', 'P30', 'P45', 'W21', 'W22', 'W25', 'W31',
       'W32', 'W48', 'W50', 'SmFan', 'SmLPC', 'SmHPC', 'phi'],
      dtype='object')


In [ ]:
import h5py
import numpy as np
import pandas as pd

# Make sure h5_path is already defined as in your train code
print("Using HDF5 file:", h5_path)

# 1. Load TEST arrays and column names from HDF5
with h5py.File(h5_path, 'r') as hdf:
    # Main TEST arrays
    A_test = hdf["A_test"][:]      # (N_test, 4)  -> ['unit', 'cycle', 'Fc', 'hs']
    T_test = hdf["T_test"][:]      # (N_test, 10) -> health modifiers
    W_test = hdf["W_test"][:]      # (N_test, 4)  -> ['alt', 'Mach', 'TRA', 'T2']
    Xs_test = hdf["X_s_test"][:]   # (N_test, 14) -> physical sensors
    Xv_test = hdf["X_v_test"][:]   # (N_test, 14) -> virtual sensors
    Y_test = hdf["Y_test"][:]      # (N_test, 1)  -> RUL target

    # Column names (same as train)
    A_var  = [x.decode("utf-8") for x in hdf["A_var"][:]]
    T_var  = [x.decode("utf-8") for x in hdf["T_var"][:]]
    W_var  = [x.decode("utf-8") for x in hdf["W_var"][:]]
    Xs_var = [x.decode("utf-8") for x in hdf["X_s_var"][:]]
    Xv_var = [x.decode("utf-8") for x in hdf["X_v_var"][:]]
gc.collect()
# 2. Stack all TEST feature blocks horizontally into one big matrix
X_test_all = np.hstack([A_test, T_test, W_test, Xs_test, Xv_test])

# 3. Build full feature name list in the same order (same as train)
feature_names = A_var + T_var + W_var + Xs_var + Xv_var

# 4. Create full TEST pandas DataFrame (includes unit, cycle, all features)
df_test = pd.DataFrame(X_test_all, columns=feature_names)

# 5. Add RUL label as a new column for TEST
df_test["RUL"] = Y_test[:, 0]

# 6. Remove constant (useless) columns in TEST (same list you used for train)
constant_cols = [
    'fan_eff_mod', 'fan_flow_mod',
    'LPC_eff_mod', 'LPC_flow_mod',
    'HPC_eff_mod', 'HPC_flow_mod',
    'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod'
]

df_test_clean = df_test.drop(columns=constant_cols)

# 7. Define feature columns (same logic: drop RUL + unit)
#feature_cols_test = [c for c in df_test_clean.columns if c not in ["RUL", "unit"]]
#feature_cols_test = [c for c in df_test_clean.columns if c not in ["RUL"]]

# ---- PANDAS TEST SET ----
#X_test_df = df_test_clean[feature_cols_test]   # pandas features (no unit, includes cycle)
X_test_df = df_test_clean
df_test_clean['target'] = np.where(df_test_clean["RUL"]>1, 1, 0)
y_test_df = df_test_clean["target"]              # pandas labels

# only take every 1000 row or where failure occurs
failure_mask = (y_test_df == 0)
row_mask = (y_test_df.reset_index().index % 1000 == 0)
y_test_df = y_test_df[failure_mask | row_mask]
X_test_df = X_test_df[failure_mask | row_mask]


# ---- NUMPY TEST SET ----
no_unit_pd_x = X_test_df.drop(labels = ['unit', 'RUL', 'target'], axis=1)
no_unit_pd_x_rf = no_unit_pd_x[rf_importances.nlargest(n_features).index[:10]]
no_unit_pd_x_svm = no_unit_pd_x[rf_importances.nlargest(n_features).index[:5]]
print(f"after feature selection: {no_unit_pd_x}")
X_test_np_rf = no_unit_pd_x_rf.to_numpy()
X_test_np_svm = no_unit_pd_x_svm.to_numpy()
y_test_np = y_test_df.to_numpy()
#values_to_keep = np.where(y_test_np == 0)[0]
#print(y_test_np)
#print(values_to_keep)
#print(values_to_keep.shape)
#indices = range(0, int(len(y_test_np)/10))
#indices = np.array(indices)
#indices = indices*10
#print(indices.shape)
#values_to_keep = np.concatenate((values_to_keep, indices))
#X_test_np = X_test_np[values_to_keep]
#y_test_np = y_test_np[values_to_keep]

print("\n=== TEST DATA SHAPES ===")
print("df_test_clean:", df_test_clean.shape)
print(f"df labels: {X_test_df.columns}")
print(f"df for np: {no_unit_pd_x.columns}")
print("X_test_df:", X_test_df.shape, "| y_test_df:", y_test_df.shape)
print("X_test_np_rf:", X_test_np_rf.shape, "| y_test_np:", y_test_np.shape)
print("X_test_np_svm:", X_test_np_svm.shape, "| y_test_np:", y_test_np.shape)
print("Test units:", df_test_clean['unit'].unique())


In [ ]:
del hdf
gc.collect()

In [ ]:
## General helper methods
def fold(x, y, i, nfolds):
    # your code
    print(f"fold {i}")
    x_len = len(y)
    val_split_start = int((i)*(x_len/nfolds))
    val_split_end = int((i+1)*(x_len/nfolds))
    if val_split_end>=x_len:
      x_val = x[val_split_start:]
      y_val = y[val_split_start:]
      x_train = x[:val_split_start]
      y_train = y[:val_split_start]
    elif val_split_start==0:
      x_val = x[:val_split_end]
      y_val = y[:val_split_end]
      x_train = x[val_split_end:]
      y_train = y[val_split_end:]
    else:
      x_val = x[val_split_start:val_split_end]
      y_val = y[val_split_start:val_split_end]
      x_train = np.append(x[:val_split_start], x[val_split_end:], axis=0)
      y_train = np.append(y[:val_split_start], y[val_split_end:], axis=0)
    return x_train, y_train, x_val, y_val

In [ ]:
from scipy import stats as st
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
import tensorflow as tf
def single_hyper_test_approach(x_train, y_train, hyperparameters, model_type):
  #shuffle training data
  train_indices = [idx for idx in range(len(y_train))]
  import random
  random.seed(0)
  random.shuffle(train_indices)
  count = 0
  x_train = x_train[train_indices]
  y_train = y_train[train_indices]
  nfolds = 4
  acc_by_fold = np.zeros(nfolds)
  #cross validation
  for num_fold in range(nfolds):
    fold_x_train, fold_y_train, fold_x_val, fold_y_val = fold(x_train, y_train, num_fold, nfolds)

    if model_type == "SVM":
      #fitted_model = SVM_fitting(torch.from_numpy(fold_x_train).to(device), torch.from_numpy(fold_y_train).to(device), hyperparameters)
      fitted_model = SVM_fitting(fold_x_train, fold_y_train, hyperparameters)
    else:
      #fitted_model = tree_fitting(torch.from_numpy(fold_x_train).to(device), torch.from_numpy(fold_y_train).to(device), hyperparameters)
      fitted_model = tree_fitting(fold_x_train, fold_y_train, hyperparameters)

    y_pred = y_predict(fitted_model, fold_x_val)
    fold_acc = calc_accuracy(y_pred, fold_y_val)
    acc_by_fold[num_fold] = fold_acc
  accuracy=0
  for acc in acc_by_fold:
    accuracy+=acc
  accuracy = accuracy/nfolds
  return accuracy
def SVM_fitting(x_train, y_train, hyperparameters):
  model = SVC(kernel=hyperparameters['kernel'], C=hyperparameters['C'], degree=hyperparameters['degree'])
  model.fit(np.asarray(x_train), np.asarray(y_train))
  return model
def tree_fitting(x_train, y_train, hyperparameters):
  model = RFC(max_depth=hyperparameters['max_depth'], min_samples_split = hyperparameters['min_sample_splits'], n_estimators=hyperparameters['n_estimators'])
  model.fit(np.asarray(x_train), np.asarray(y_train))
  return model
def y_predict(fitted_model, test_data):
  return fitted_model.predict(test_data)
def calc_accuracy(y_predict, y):
    # your code
    num_accurate = 0
    for y_loc in range(len(y_predict)):
      if y_predict[y_loc] == y[y_loc]:
        num_accurate += 1
    acc = num_accurate/len(y_predict)
    return acc

In [ ]:
# these are features importances
print(rf_importances)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ! pip install --upgrade pytorch-forecasting
# ! pip install lightning
# ! pip install pytorch-lightning
# ! pip install torchrl

In [ ]:

# import torch
# from pytorch_forecasting.models.temporal_fusion_transformer._tft import TemporalFusionTransformer
# from pytorch_forecasting.data.timeseries._timeseries import TimeSeriesDataSet
# from torch.nn import CrossEntropyLoss
# from lightning.pytorch.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
# from torchrl.record.loggers.csv import CSVLogger
# from lightning.pytorch import Trainer
# ## FORWARD FILL

# def tft(x_train, y_train, x_test):
#   print(x_train)
#   x_train.apply(pd.to_numeric)
#   y_train.apply(pd.to_numeric)
#   x_test.apply(pd.to_numeric)
#   x_train['cycle'] = x_train['cycle'].astype(int)
#   x_test['cycle'] = x_test['cycle'].astype(int)
#   print(f"checking columns : {x_train.columns}")
#   print(f"checking rows: {x_train.index.unique()}")
#   training = TimeSeriesDataSet(
#       x_train,
#       time_idx="cycle",
#       target="RUL",
#       group_ids=['unit'],

#       min_encoder_length=50, # Make sure each sequence has atleast 50 sequences
#       max_encoder_length=50,
#       min_prediction_length=1,
#       max_prediction_length=1,

#       static_categoricals=[],
#       static_reals=["unit"],
#       time_varying_known_categoricals=[],
#       time_varying_known_reals=[ "cycle",],
#       time_varying_unknown_reals=[ "Fc", "hs", "alt", "Mach", "TRA", "T2","T24", "T30", "T48", "T50", "P15", "P2", "P21", "P24","Ps30", "P40", "P50", "Nf", "Nc", "Wf","T40", "P30", "P45", "W21", "W22", "W25", "W31", "W32","W48", "W50", "SmFan", "SmLPC", "SmHPC", "phi",],

#       # target_normalizer=NaNLabelEncoder(add_nan=True),
#       #categorical_encoders=categorical_encoders,
#       add_relative_time_idx=True,
#       add_target_scales=False,
#       add_encoder_length=True,
#       allow_missing_timesteps=True
#   )
#   train_dataloader = training.to_dataloader(
#     train=True, batch_size=120, num_workers=3)

#   validation = TimeSeriesDataSet.from_dataset(
#       training, x_test, predict=True, stop_randomization=True)

#   val_dataloader = validation.to_dataloader(
#       train=False, batch_size=120 * 10, num_workers=3)
#   tft = TemporalFusionTransformer.from_dataset(
#       training,
#       learning_rate=0.01,
#       hidden_size=128,
#       lstm_layers=2, # avoid increasing
#       attention_head_size=8,
#       dropout=0.1,
#       hidden_continuous_size=32,
#       loss=CrossEntropyLoss(),  # Key difference for classification
#       output_size=2,
#       optimizer='ranger', # try 'adamW'
#       share_single_variable_networks=True,
#       causal_attention=False,
#   )
#   early_stop_callback = EarlyStopping(
#     monitor="val_loss", min_delta=1e-4, patience=16, verbose=True, mode="min"
#   )

#   checkpoint_callback = ModelCheckpoint(
#       monitor="val_loss",
#       save_top_k=5,
#       every_n_epochs=1,
#       dirpath="checkpoints/",
#       filename="tft-epoch-{epoch:02d}-val_loss-{val_loss:.4f}",
#   )

#   csv_logger = CSVLogger("logs/", name="tft")
#   trainer = Trainer(
#     max_epochs=400, # ranger optimizer is slow
#     accelerator="auto",
#     devices="auto",
#     enable_model_summary=True,
#     gradient_clip_val=0.1,
#     callbacks=[checkpoint_callback, early_stop_callback],
#     logger=csv_logger,
#     log_every_n_steps=1,
#     check_val_every_n_epoch=1,
#     enable_checkpointing=True
#   )

#   trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
#   pred = tft.predict(validation)
#   #label_idx = int(pred[0][0][1:].argmax() + 1)

#   # Convert to gesture name
#   # gesture = reverse_map[label_idx]

#   # # Store in results list
#   # results.append({"sequence_id": seq, "gesture": gesture})
#   # inverse_map = {v: k for k, v in categorical_encoders['gesture']._mapping.items()}
#   # predicted_label = inverse_map[predicted_class_id]

#   return pred

In [ ]:
def hypertuning_svm_rf(x_train_rf, x_train_svm, y_train):
  best_svm_acc = 0
  best_rf_acc = 0
  best_svm_hypers = {}
  best_rf_hypers = {}
  kernel = ['rbf'] #, 'linear', 'poly']
  C = [1] # , 2]
  degree = [2] #, 3]
  for kernel_ind in kernel:
    for c_ind in C:
      if kernel_ind == 'poly':
        for degree_ind in degree:
          hyperparameters = {
          'kernel': kernel_ind,
          'C': c_ind,
          'degree': degree
          }
          accuracy = single_hyper_test_approach(x_train_svm, y_train, hyperparameters, "SVM")
          print("SVM:")
          print(f"{hyperparameters}, accuracy: {accuracy}")
          if best_svm_acc < accuracy:
            best_svm_acc = accuracy
            best_svm_hypers = hyperparameters
      else:
        hyperparameters = {
        'kernel': kernel_ind,
        'C': c_ind,
        'degree': 1
        }
        accuracy = single_hyper_test_approach(x_train_svm, y_train, hyperparameters, "SVM")
        print("SVM:")
        print(f"{hyperparameters}, accuracy: {accuracy}")
        if best_svm_acc < accuracy:
          best_svm_acc = accuracy
          best_svm_hypers = hyperparameters

  n_estimators = [50, 100]
  min_sample_splits = [2, 3]
  max_depth = [2, 4]
  for n_est in n_estimators:
    for min_split in min_sample_splits:
      for depth in max_depth:
        hyperparameters = {
          'n_estimators': n_est,
          'min_sample_splits': min_split,
          'max_depth': depth
        }
        accuracy = single_hyper_test_approach(x_train_rf, y_train, hyperparameters, "RF")
        print("RF:")
        print(f"n_estimators: {n_est}, min_sample_splits: {min_split}, max_depth: {depth}, accuracy: {accuracy}")
        if best_rf_acc < accuracy:
          best_rf_acc = accuracy
          best_rf_hypers = hyperparameters
  return best_rf_hypers, best_svm_hypers


In [ ]:
x_train_rf = X_train_np_rf
x_train_svm = X_train_np_svm
y_train = y_train_np
x_test_rf = X_test_np_rf
x_test_svm = X_test_np_svm
y_test = y_test_np

best_rf_hypers, best_svm_hypers = hypertuning_svm_rf(x_train_rf, x_train_svm, y_train)

fitted_rf = tree_fitting(x_train_rf, y_train, best_rf_hypers)
rf_prediction = y_predict(fitted_rf, x_test_rf)
rf_accuracy = calc_accuracy(rf_prediction, y_test)
print(f"rf overall accuracy: {rf_accuracy}")

fitted_svm = SVM_fitting(x_train_svm, y_train, best_svm_hypers)
svm_prediction = y_predict(fitted_svm, x_test_svm)
svm_accuracy = calc_accuracy(svm_prediction, y_test)
print(f"svm overall accuracy: {svm_accuracy}")

In [ ]:
rf_train_pred = y_predict(fitted_rf, x_train_rf)
rf_train_accuracy = calc_accuracy(rf_train_pred, y_train)

svm_train_pred = y_predict(fitted_svm, x_train_svm)
svm_train_accuracy = calc_accuracy(svm_train_pred, y_train)

models = ["RF", "SVM"]
train_accs = [rf_train_accuracy, svm_train_accuracy]
test_accs = [rf_accuracy, svm_accuracy]
x = np.arange(len(models))
width = 0.35
plt.figure(figsize=(6,4))
plt.bar(x - width/2, train_accs, width, label='Training')
plt.bar(x + width/2, test_accs, width, label='Testing')
plt.xticks(x, models)
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()
plt.tightlayout()
plt.show()

# RF confusion matrix
rf_cm = confusion_matrix(y_test, rf_prediction)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=rf_cm, display_labels=[0, 1])
plt.figure(figsize=(4,4))
disp_rf.plot(values_format="d")
plt.title("Random Forest - Test Confusion Matrix")
plt.show()

# SVM confusion matrix
svm_cm = confusion_matrix(y_test, svm_prediction)
disp_svm = ConfusionMatrixDisplay(confusion_matrix=svm_cm, display_labels=[0, 1])
plt.figure(figsize=(4,4))
disp_svm.plot(values_format="d")
plt.title("SVM - Test Confusion Matrix")
plt.show()

In [ ]:
# x_train = X_train_df
# y_train = y_train_df
# x_test = df_test_clean
# tft_prediction = tft(x_train, y_train, x_test)
# tft_accuracy = calc_accuracy(tft_prediction, y_test)
# print(f"tft overall accuracy: {tft_accuracy}")

In [ ]:
# svm output
fold 0
fold 1
fold 2
fold 3
SVM:
{'kernel': 'rbf', 'C': 1, 'degree': 1}, accuracy: 0.996842253175913
fold 0
fold 1
fold 2
fold 3
SVM:
{'kernel': 'rbf', 'C': 2, 'degree': 1}, accuracy: 0.9970313740866987


In [ ]:
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 2, max_depth: 2, accuracy: 0.9982635261409849
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 2, max_depth: 4, accuracy: 0.9988194265371614
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 2, max_depth: 8, accuracy: 0.9999426907166977
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 3, max_depth: 2, accuracy: 0.9980572129575725
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 3, max_depth: 4, accuracy: 0.9988194265371614
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 3, max_depth: 8, accuracy: 0.9999340943012118
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 4, max_depth: 2, accuracy: 0.998358086350049
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 4, max_depth: 4, accuracy: 0.9988194265371614
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 50, min_sample_splits: 4, max_depth: 8, accuracy: 0.9999169015030835
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 2, max_depth: 2, accuracy: 0.9983151028603336
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 2, max_depth: 4, accuracy: 0.9988194265371614
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 2, max_depth: 8, accuracy: 0.9999484216384593
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 3, max_depth: 2, accuracy: 0.9979225362961782
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 3, max_depth: 4, accuracy: 0.9988194265371614
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 3, max_depth: 8, accuracy: 0.9999541525602207
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 4, max_depth: 2, accuracy: 0.9978480343132794
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 4, max_depth: 4, accuracy: 0.9988194265371614
fold 0
fold 1
fold 2
fold 3
RF:
n_estimators: 100, min_sample_splits: 4, max_depth: 8, accuracy: 0.99995128709934